In [505]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score , mean_absolute_error, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, RandomizedSearchCV
from category_encoders import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.impute import SimpleImputer
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVR
import seaborn as sns
import pandas as pd
import numpy as np
import ast
import pickle
import warnings
warnings.simplefilter("ignore")


from sklearn.feature_selection import mutual_info_regression

In [506]:
test = pd.read_csv(r"C:\Users\Admin\Desktop\All files\juypternotebooks files\ijaz task\test_regression.csv")
df = pd.read_csv(r"C:\Users\Admin\Desktop\All files\juypternotebooks files\ijaz task\train_regression.csv")
pd.set_option('display.max_columns', None)
df.head()

,id,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5.387292e+07,366516391,4/9/2018,"Chicago, IL",within an hour,96%,95%,f,Back of the Yards,11,13,"['email', 'phone']",t,t,New City,41.794240,-87.661380,Private room in rental unit,Private room,1,1 shared bath,1.0,$30.00,32,1125,32,32,1125,1125,32.0,1125.0,t,30,60,90,365,18,0,0,6/16/2019,8/12/2022,4.94,5.00,4.89,5.00,5.00,4.28,4.83,f,9,1,8,0,0.33
1,3.926966e+07,50276775,12/23/2014,NaN,within an hour,100%,97%,f,Clearwater Beach,1141,2346,"['email', 'phone']",t,t,Near North Side,41.892749,-87.620711,Room in hotel,Private room,12,3 baths,3.0,"$1,247.00",32,365,32,32,365,365,32.0,365.0,t,25,49,66,135,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,58,0,58,0,NaN
2,5.564810e+17,681680651,3/8/2020,NaN,within an hour,100%,100%,f,Lake View East,1,1,['phone'],t,t,Lake View,41.948160,-87.652309,Entire rental unit,Entire home/apt,6,1 bath,3.0,$214.00,2,45,2,2,45,45,2.0,45.0,t,0,0,0,0,14,14,1,7/21/2023,12/3/2023,4.87,5.00,5.00,5.00,5.00,4.93,4.80,f,1,1,0,0,2.78
3,5.239728e+07,94345291,10/22/2015,"Chicago, IL",within an hour,100%,98%,f,Bucktown,55,58,"['email', 'phone', 'work_email']",t,t,Hyde Park,41.801030,-87.597160,Entire rental unit,Entire home/apt,2,1 bath,1.0,$59.00,2,180,2,2,180,180,2.0,180.0,t,23,53,83,355,13,13,3,9/2/2023,12/6/2023,4.08,4.08,4.00,4.38,4.23,4.54,3.85,t,55,55,0,0,3.61
4,7.459490e+17,790998106,4/4/2021,"Chicago, IL",within an hour,100%,100%,t,Near South Side,74,75,"['email', 'phone']",t,t,Near South Side,41.855770,-87.624130,Entire rental unit,Entire home/apt,6,2 baths,3.0,$151.00,2,365,2,2,1125,1125,2.0,1125.0,t,0,0,0,0,64,42,3,5/6/2022,12/6/2023,4.80,4.74,4.75,4.83,4.85,4.74,4.72,t,74,74,0,0,3.24


In [507]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 54 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            5000 non-null   float64
 1   host_id                                       5000 non-null   int64  
 2   host_since                                    5000 non-null   object 
 3   host_location                                 4042 non-null   object 
 4   host_response_time                            4582 non-null   object 
 5   host_response_rate                            4582 non-null   object 
 6   host_acceptance_rate                          4709 non-null   object 
 7   host_is_superhost                             4977 non-null   object 
 8   host_neighbourhood                            4873 non-null   object 
 9   host_listings_count                           5000 non-null   i

<h1>Data Cleaning</h1>

In [508]:
df.drop(columns = ['id'], inplace = True)

In [509]:
df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float)

In [510]:
df['host_location'] = df['host_location'].str.replace(',', '')
df['host_response_rate'] = df['host_response_rate'].str.replace('%', '')
df['host_acceptance_rate'] = df['host_acceptance_rate'].str.replace('%', '')

test['host_location'] = test['host_location'].str.replace(',', '')
test['host_response_rate'] = test['host_response_rate'].str.replace('%', '')
test['host_acceptance_rate'] = test['host_acceptance_rate'].str.replace('%', '')

In [511]:
df['host_since'] = pd.to_datetime(df['host_since'], errors='coerce')
df['first_review'] = pd.to_datetime(df['first_review'], errors='coerce')
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')


df['host_since_year'] = df['host_since'].dt.year
df['host_since_month'] = df['host_since'].dt.month
df['host_since_day'] = df['host_since'].dt.day

df['first_review_year'] = df['first_review'].dt.year
df['first_review_month'] = df['first_review'].dt.month
df['first_review_day'] = df['first_review'].dt.day

df['last_review_year'] = df['last_review'].dt.year
df['last_review_month'] = df['last_review'].dt.month
df['last_review_day'] = df['last_review'].dt.day


df = df.drop(columns=['host_since', 'first_review', 'last_review'])

In [512]:
df.head()

,host_id,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_since_year,host_since_month,host_since_day,first_review_year,first_review_month,first_review_day,last_review_year,last_review_month,last_review_day
0,366516391,Chicago IL,within an hour,96,95,f,Back of the Yards,11,13,"['email', 'phone']",t,t,New City,41.794240,-87.661380,Private room in rental unit,Private room,1,1 shared bath,1.0,30.0,32,1125,32,32,1125,1125,32.0,1125.0,t,30,60,90,365,18,0,0,4.94,5.00,4.89,5.00,5.00,4.28,4.83,f,9,1,8,0,0.33,2018,4,9,2019.0,6.0,16.0,2022.0,8.0,12.0
1,50276775,NaN,within an hour,100,97,f,Clearwater Beach,1141,2346,"['email', 'phone']",t,t,Near North Side,41.892749,-87.620711,Room in hotel,Private room,12,3 baths,3.0,1247.0,32,365,32,32,365,365,32.0,365.0,t,25,49,66,135,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,58,0,58,0,NaN,2014,12,23,NaN,NaN,NaN,NaN,NaN,NaN
2,681680651,NaN,within an hour,100,100,f,Lake View East,1,1,['phone'],t,t,Lake View,41.948160,-87.652309,Entire rental unit,Entire home/apt,6,1 bath,3.0,214.0,2,45,2,2,45,45,2.0,45.0,t,0,0,0,0,14,14,1,4.87,5.00,5.00,5.00,5.00,4.93,4.80,f,1,1,0,0,2.78,2020,3,8,2023.0,7.0,21.0,2023.0,12.0,3.0
3,94345291,Chicago IL,within an hour,100,98,f,Bucktown,55,58,"['email', 'phone', 'work_email']",t,t,Hyde Park,41.801030,-87.597160,Entire rental unit,Entire home/apt,2,1 bath,1.0,59.0,2,180,2,2,180,180,2.0,180.0,t,23,53,83,355,13,13,3,4.08,4.08,4.00,4.38,4.23,4.54,3.85,t,55,55,0,0,3.61,2015,10,22,2023.0,9.0,2.0,2023.0,12.0,6.0
4,790998106,Chicago IL,within an hour,100,100,t,Near South Side,74,75,"['email', 'phone']",t,t,Near South Side,41.855770,-87.624130,Entire rental unit,Entire home/apt,6,2 baths,3.0,151.0,2,365,2,2,1125,1125,2.0,1125.0,t,0,0,0,0,64,42,3,4.80,4.74,4.75,4.83,4.85,4.74,4.72,t,74,74,0,0,3.24,2021,4,4,2022.0,5.0,6.0,2023.0,12.0,6.0


In [513]:
df['host_verifications'] = df['host_verifications'].apply(ast.literal_eval)


verification_types = ['email_verification', 'phone_verification', 'work_email_verification']


for verification in verification_types:
    df[verification] = df['host_verifications'].apply(lambda x: 1 if verification in x else 0)


df = df.drop('host_verifications', axis=1)

In [514]:
df["host_is_superhost"] = df["host_is_superhost"].replace({'t': 1, 'f': 0})
df["host_has_profile_pic"] = df["host_has_profile_pic"].replace({'t': 1, 'f': 0})
df["host_identity_verified"] = df["host_identity_verified"].replace({'t': 1, 'f': 0})
df["has_availability"] = df["has_availability"].replace({'t': 1, 'f': 0})
df["instant_bookable"] = df["has_availability"].replace({'t': 1, 'f': 0})

In [515]:
df.head()

,host_id,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_since_year,host_since_month,host_since_day,first_review_year,first_review_month,first_review_day,last_review_year,last_review_month,last_review_day,email_verification,phone_verification,work_email_verification
0,366516391,Chicago IL,within an hour,96,95,0.0,Back of the Yards,11,13,1,1,New City,41.794240,-87.661380,Private room in rental unit,Private room,1,1 shared bath,1.0,30.0,32,1125,32,32,1125,1125,32.0,1125.0,1,30,60,90,365,18,0,0,4.94,5.00,4.89,5.00,5.00,4.28,4.83,1,9,1,8,0,0.33,2018,4,9,2019.0,6.0,16.0,2022.0,8.0,12.0,0,0,0
1,50276775,NaN,within an hour,100,97,0.0,Clearwater Beach,1141,2346,1,1,Near North Side,41.892749,-87.620711,Room in hotel,Private room,12,3 baths,3.0,1247.0,32,365,32,32,365,365,32.0,365.0,1,25,49,66,135,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,58,0,58,0,NaN,2014,12,23,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
2,681680651,NaN,within an hour,100,100,0.0,Lake View East,1,1,1,1,Lake View,41.948160,-87.652309,Entire rental unit,Entire home/apt,6,1 bath,3.0,214.0,2,45,2,2,45,45,2.0,45.0,1,0,0,0,0,14,14,1,4.87,5.00,5.00,5.00,5.00,4.93,4.80,1,1,1,0,0,2.78,2020,3,8,2023.0,7.0,21.0,2023.0,12.0,3.0,0,0,0
3,94345291,Chicago IL,within an hour,100,98,0.0,Bucktown,55,58,1,1,Hyde Park,41.801030,-87.597160,Entire rental unit,Entire home/apt,2,1 bath,1.0,59.0,2,180,2,2,180,180,2.0,180.0,1,23,53,83,355,13,13,3,4.08,4.08,4.00,4.38,4.23,4.54,3.85,1,55,55,0,0,3.61,2015,10,22,2023.0,9.0,2.0,2023.0,12.0,6.0,0,0,0
4,790998106,Chicago IL,within an hour,100,100,1.0,Near South Side,74,75,1,1,Near South Side,41.855770,-87.624130,Entire rental unit,Entire home/apt,6,2 baths,3.0,151.0,2,365,2,2,1125,1125,2.0,1125.0,1,0,0,0,0,64,42,3,4.80,4.74,4.75,4.83,4.85,4.74,4.72,1,74,74,0,0,3.24,2021,4,4,2022.0,5.0,6.0,2023.0,12.0,6.0,0,0,0


<h1>Handling Null Values</h1>

In [516]:
null_counts = df.isnull().sum()

columns_with_nulls = null_counts[null_counts > 0]
columns_with_nulls

host_location                   958
host_response_time              418
host_response_rate              418
host_acceptance_rate            291
host_is_superhost                23
host_neighbourhood              127
bathrooms_text                    4
beds                             48
review_scores_rating           1041
review_scores_accuracy         1041
review_scores_cleanliness      1042
review_scores_checkin          1041
review_scores_communication    1042
review_scores_location         1041
review_scores_value            1041
reviews_per_month              1042
first_review_year              1042
first_review_month             1042
first_review_day               1042
last_review_year               1042
last_review_month              1042
last_review_day                1042
dtype: int64

In [517]:
null_counts = df.isnull().sum()

columns_with_nulls = null_counts[null_counts > 0].index

df_with_nulls = df[columns_with_nulls]
df_with_nulls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   host_location                4042 non-null   object 
 1   host_response_time           4582 non-null   object 
 2   host_response_rate           4582 non-null   object 
 3   host_acceptance_rate         4709 non-null   object 
 4   host_is_superhost            4977 non-null   float64
 5   host_neighbourhood           4873 non-null   object 
 6   bathrooms_text               4996 non-null   object 
 7   beds                         4952 non-null   float64
 8   review_scores_rating         3959 non-null   float64
 9   review_scores_accuracy       3959 non-null   float64
 10  review_scores_cleanliness    3958 non-null   float64
 11  review_scores_checkin        3959 non-null   float64
 12  review_scores_communication  3958 non-null   float64
 13  review_scores_loca

In [518]:
df['host_response_rate'] = df['host_response_rate'].astype(float)

In [519]:
df['host_is_superhost'] = df['host_is_superhost'].astype(float)

In [520]:
mode_values = df[['host_location', 'host_response_time', 'host_is_superhost', 'host_acceptance_rate', 'host_response_rate', 'host_is_superhost', 'host_neighbourhood', 'bathrooms_text']].mode().iloc[0]
df.fillna(mode_values, inplace=True)

In [521]:
df[['beds', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month', 'first_review_year', 'first_review_month', 'first_review_day', 'last_review_year', 'last_review_month', 'last_review_day']] = df[['beds', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month', 'first_review_year', 'first_review_month', 'first_review_day', 'last_review_year', 'last_review_month', 'last_review_day']].fillna(df[['beds', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month', 'first_review_year', 'first_review_month', 'first_review_day', 'last_review_year', 'last_review_month', 'last_review_day']].median())


In [522]:
df.drop(columns=['host_id'], inplace=True)

<h1>Label + One Hot encoding</h1>

In [523]:
df.select_dtypes(include = 'number').columns

Index(['host_response_rate', 'host_is_superhost', 'host_listings_count',
       'host_total_listings_count', 'host_has_profile_pic',
       'host_identity_verified', 'latitude', 'longitude', 'accommodates',
       'beds', 'price', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'number_of_reviews_ltm',
       'number_of_reviews_l30d', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private

In [524]:
from sklearn.preprocessing import OneHotEncoder

one_hot_columns = ['room_type', 'host_response_time']


one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
one_hot_encoded = one_hot_encoder.fit_transform(df[one_hot_columns])


one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=one_hot_encoder.get_feature_names_out(one_hot_columns))


df = df.drop(columns=one_hot_columns)
df = pd.concat([df, one_hot_encoded_df], axis=1)


In [525]:
label_columns = ['host_location', 'host_neighbourhood', 'neighbourhood_cleansed', 'property_type', 'bathrooms_text']


label_encoders = {}
for column in label_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le


In [526]:
df['capacity_to_bed_ratio'] = df['accommodates'] / df['beds']


<h1>Feature Engineering </h1>

<h2>Correlation Coefficients</h2>

In [527]:
corr = df.corr()
target_corr = corr['price'].sort_values(ascending=False)
print(target_corr[:50])

price                                           1.000000
accommodates                                    0.086968
beds                                            0.082688
bathrooms_text                                  0.058149
room_type_Entire home/apt                       0.031616
host_neighbourhood                              0.028112
review_scores_location                          0.021711
latitude                                        0.019247
review_scores_cleanliness                       0.017257
review_scores_rating                            0.016616
review_scores_accuracy                          0.015419
review_scores_checkin                           0.012558
review_scores_communication                     0.012031
longitude                                       0.009401
host_response_time_within an hour               0.007845
review_scores_value                             0.007339
calculated_host_listings_count_private_rooms    0.007241
neighbourhood_cleansed         

In [528]:
a = target_corr[23:].index

<h1>Dropping columns</h1>

In [529]:
columns_to_remove = ['has_availability', 'instant_bookable', 'maximum_maximum_nights',
       'host_total_listings_count', 'maximum_nights_avg_ntm',
       'minimum_maximum_nights', 'host_response_time_a few days or more',
       'host_response_time_within a day', 'room_type_Hotel room',
       'host_listings_count', 'first_review_month', 'host_since_year',
       'minimum_minimum_nights', 'minimum_nights', 'number_of_reviews',
       'maximum_nights', 'host_location',
       'calculated_host_listings_count_shared_rooms', 'room_type_Shared room',
       'host_is_superhost', 'calculated_host_listings_count',
       'host_response_time_within a few hours',
       'calculated_host_listings_count_entire_homes', 'maximum_minimum_nights',
       'host_since_month', 'host_since_day', 'minimum_nights_avg_ntm',
       'first_review_day', 'availability_30', 'last_review_day',
       'first_review_year', 'availability_60', 'number_of_reviews_l30d',
       'availability_365', 'last_review_month', 'reviews_per_month',
       'number_of_reviews_ltm', 'availability_90', 'property_type',
       'room_type_Private room', 'host_identity_verified',
       'email_verification', 'phone_verification', 'work_email_verification', 'last_review_year']

df.drop(columns = columns_to_remove, inplace = True)

In [530]:
X = df.drop(columns = 'price')
y = df['price']

In [531]:
mi = mutual_info_regression(X, y)


mi_series = pd.Series(mi, index=X.columns, name='Mutual Information')
mi_series = mi_series.sort_values(ascending=False)
mi_series[11:].index

Index(['review_scores_rating', 'review_scores_communication',
       'host_response_rate', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_location', 'review_scores_value', 'host_has_profile_pic',
       'host_response_time_within an hour'],
      dtype='object')

In [532]:
columns = ['host_response_rate', 'review_scores_rating', 'review_scores_location',
       'review_scores_cleanliness', 'review_scores_communication',
       'review_scores_accuracy', 'review_scores_checkin',
       'review_scores_value', 'host_response_time_within an hour',
       'host_has_profile_pic']
X.drop(columns = columns, inplace = True)

In [566]:
X.head()

,host_acceptance_rate,host_neighbourhood,neighbourhood_cleansed,latitude,longitude,accommodates,bathrooms_text,beds,calculated_host_listings_count_private_rooms,room_type_Entire home/apt,capacity_to_bed_ratio
0,95,9,49,41.794240,-87.661380,1,3,1.0,8,0.0,1.0
1,97,35,46,41.892749,-87.620711,12,13,3.0,58,0.0,4.0
2,100,87,37,41.948160,-87.652309,6,1,3.0,0,1.0,2.0
3,98,22,33,41.801030,-87.597160,2,1,1.0,0,1.0,2.0
4,100,111,47,41.855770,-87.624130,6,9,3.0,0,1.0,2.0


<h2>Removing outliers</h2>

In [534]:
X_numeric = X.select_dtypes(include=[np.number])


Q1 = X_numeric.quantile(0.25)
Q3 = X_numeric.quantile(0.75)


IQR = Q3 - Q1


lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


outliers = (X_numeric < lower_bound) | (X_numeric > upper_bound)


X_cleaned = X[~outliers.any(axis=1)]
y_cleaned = y[~outliers.any(axis=1)]

In [535]:
X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

<h1> Linear Regression </h1>

<h3>
    StandardScaler
</h3>

In [536]:
scaler = StandardScaler()

In [537]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [538]:
def try_model1(model):
    model.fit(X_train_scaled, y_train)
    train_preds = model.predict(X_train_scaled)
    test_preds = model.predict(X_test_scaled)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
    test_rmse = np.sqrt(mean_squared_error(test_preds, y_test))
    print("train_rmse is:", train_rmse)
    print("test_rmse is:", test_rmse)

In [539]:
try_model1(LinearRegression())

train_rmse is: 2486.8872807632547
test_rmse is: 246.74399231090334


<h3>MinMaxScaler</h3>

In [540]:
scaler_min_max = MinMaxScaler()

In [541]:
X_train_scaled_min_max = scaler_min_max.fit_transform(X_train)
X_test_scaled_min_max = scaler_min_max.transform(X_test)

In [542]:
def try_model2(model):
    model.fit(X_train_scaled_min_max, y_train)
    train_preds = model.predict(X_train_scaled_min_max)
    test_preds = model.predict(X_test_scaled_min_max)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
    test_rmse = np.sqrt(mean_squared_error(test_preds, y_test))
    print("train_rmse is:", train_rmse)
    print("test_rmse is:", test_rmse)


In [543]:
try_model2(LinearRegression())

train_rmse is: 2486.887280763255
test_rmse is: 246.74399231090257


<h1> Decision Tree with Min scaler and Standard scaler </h1>

<h3> With standard scaler </h3>

In [544]:
def try_model3(model):
    model.fit(X_train_scaled, y_train)
    train_preds = model.predict(X_train_scaled)
    test_preds = model.predict(X_test_scaled)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
    test_rmse = np.sqrt(mean_squared_error(test_preds, y_test))
    print("train_rmse is:", train_rmse)
    print("test_rmse is:", test_rmse)

In [545]:
try_model3(DecisionTreeRegressor())

train_rmse is: 12.50172512718551
test_rmse is: 141.25741435190523


In [546]:
reg = DecisionTreeRegressor(
    criterion='squared_error',          # Mean squared error (default). Alternatives: 'friedman_mse', 'mae'
    splitter='best',          # Strategy used to choose the split ('best' or 'random')
    max_depth=6,              # Maximum depth of the tree
    min_samples_split=10,     # Minimum number of samples required to split an internal node
    min_samples_leaf=5,       # Minimum number of samples required to be at a leaf node
    max_features='sqrt',      # Number of features to consider when looking for the best split ('sqrt', 'log2', or int)
    max_leaf_nodes=20,        # Maximum number of leaf nodes
    min_impurity_decrease=0.01, # Minimum impurity decrease required to split a node
    random_state=42           # Random seed for reproducibility (optional)
)

try_model3(reg)

train_rmse is: 2235.086965735009
test_rmse is: 80.14870744616326


<h3>
    with Min Max scaler
</h3>

In [547]:
def try_model4(model):
    model.fit(X_train_scaled_min_max, y_train)
    train_preds = model.predict(X_train_scaled_min_max)
    test_preds = model.predict(X_test_scaled)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
    test_rmse = np.sqrt(mean_squared_error(test_preds, y_test))
    print("train_rmse is:", train_rmse)
    print("test_rmse is:", test_rmse)

In [548]:
reg = DecisionTreeRegressor(
    criterion='squared_error',          # Mean squared error (default). Alternatives: 'friedman_mse', 'mae'
    splitter='best',          # Strategy used to choose the split ('best' or 'random')
    max_depth=6,              # Maximum depth of the tree
    min_samples_split=10,     # Minimum number of samples required to split an internal node
    min_samples_leaf=5,       # Minimum number of samples required to be at a leaf node
    max_features='sqrt',      # Number of features to consider when looking for the best split ('sqrt', 'log2', or int)
    max_leaf_nodes=20,        # Maximum number of leaf nodes
    min_impurity_decrease=0.01, # Minimum impurity decrease required to split a node
    random_state=42           # Random seed for reproducibility (optional)
)

try_model4(reg)

train_rmse is: 2235.086965735009
test_rmse is: 5659.756809601812


<h1>XGBRegressor</h1>

<h3>
    with standard scaler
</h3>

In [549]:
def try_model5(model):
    model.fit(X_train_scaled, y_train)
    train_preds = model.predict(X_train_scaled)
    test_preds = model.predict(X_test_scaled)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
    test_rmse = np.sqrt(mean_squared_error(test_preds, y_test))
    print("train_rmse is:", train_rmse)
    print("test_rmse is:", test_rmse)

In [550]:
model_xgb = XGBRegressor(n_estimators = 1000, random_state = 42)

In [551]:
try_model5(model_xgb)

train_rmse is: 12.530011007124413
test_rmse is: 145.34200686495646


<h3>with min max scaler</h3>

In [552]:
def try_model6(model):
    model.fit(X_train_scaled_min_max, y_train)
    train_preds = model.predict(X_train_scaled_min_max)
    test_preds = model.predict(X_test_scaled_min_max)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
    test_rmse = np.sqrt(mean_squared_error(test_preds, y_test))
    print("train_rmse is:", train_rmse)
    print("test_rmse is:", test_rmse)

In [553]:
try_model6(model_xgb)

train_rmse is: 12.530011007124413
test_rmse is: 145.34200686495646


<h1> Random Forest </h1>

<h3>Random Forest with standard scaler</h3>

In [554]:
def try_model7(model):
    model.fit(X_train_scaled, y_train)
    train_preds = model.predict(X_train_scaled)
    test_preds = model.predict(X_test_scaled)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
    test_rmse = np.sqrt(mean_squared_error(test_preds, y_test))
    print("train_rmse is:", train_rmse)
    print("test_rmse is:", test_rmse)

In [555]:
model_forest = RandomForestRegressor(n_estimators=1000, random_state=42)
try_model7(model_forest)

train_rmse is: 961.1833287598312
test_rmse is: 123.10543697076967


<h3>Random Forest with min max scaler</h3>

In [576]:
def try_model8(model):
    model.fit(X_train_scaled_min_max, y_train)
    train_preds = model.predict(X_train_scaled_min_max)
    test_preds = model.predict(X_test_scaled_min_max)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
    test_rmse = np.sqrt(mean_squared_error(test_preds, y_test))
    print("train_rmse is:", train_rmse)
    print("test_rmse is:", test_rmse)

In [577]:
model_forest = RandomForestRegressor(n_estimators=1000, 
                                     max_depth=100, 
                                     min_samples_split=5, 
                                     random_state=42)

try_model8(model_forest)

train_rmse is: 1265.595413837407
test_rmse is: 250.19126479847188


<h1> Ridge Model </h1>

<h3>Ridge model with standard scaler</h3>

In [580]:
def try_model9(model):
    model.fit(X_train_scaled, y_train)
    train_preds = model.predict(X_train_scaled)
    test_preds = model.predict(X_test_scaled)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
    test_rmse = np.sqrt(mean_squared_error(test_preds, y_test))
   
    print("train_rmse is:", train_rmse)
    print("test_rmse is:", test_rmse)

In [581]:
model = Ridge()
param_grid = {
    'alpha': [0.1, 1, 10, 100, 1000]  
}


grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)

try_model9(grid_search.fit(X_train_scaled, y_train))

train_rmse is: 2490.439859051328
test_rmse is: 161.31593539578856


<h3>Ridge model with min max scaler</h3>

In [578]:
def try_model10(model):
    model.fit(X_train_scaled_min_max, y_train)
    train_preds = model.predict(X_train_scaled_min_max)
    test_preds = model.predict(X_test_scaled_min_max)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
    test_rmse = np.sqrt(mean_squared_error(test_preds, y_test))
    train_r2 = r2_score(y_train, train_preds)
    test_r2 = r2_score(y_test, test_preds)
    
    print("train_rmse is:", train_rmse)
    print("test_rmse is:", test_rmse)

In [579]:
try_model10(grid_search.fit(X_train_scaled, y_train))

train_rmse is: 2498.7925680314006
test_rmse is: 112.83259508718339


<h1>SVM</h1>

<h3>Ridge model with standard scaler</h3>

In [562]:
def try_model11(model):
    model.fit(X_train_scaled, y_train)
    train_preds = model.predict(X_train_scaled)
    test_preds = model.predict(X_test_scaled)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
    test_rmse = np.sqrt(mean_squared_error(test_preds, y_test))
    print("train_rmse is:", train_rmse)
    print("test_rmse is:", test_rmse)

<h1>LOWEST RMSE</h1>

In [563]:
svr_regressor = SVR(kernel='rbf', C=100, epsilon=0.1)
try_model11(svr_regressor)

train_rmse is: 2499.558064991975
test_rmse is: 76.46148180901162
